# Generating Embeddings for RAG

In [3]:
import sys
print(sys.executable)

/Users/zsk4gm/Desktop/resilience_education/env/bin/python3


# Imports

In [4]:
import os
import json

### Examine a JSON

In [5]:
# Define the path to the JSON files
# Get the current directory
current_directory = os.getcwd()

# Create the file path to the 'json_files_updated' folder
json_files_updated_path = os.path.join(current_directory, 'json_files_updated')

# Print the path to verify
print(json_files_updated_path)

/Users/zsk4gm/Desktop/resilience_education/json_files_updated


In [6]:
json_exfile_updated_path = os.path.join(json_files_updated_path, '1-608.json')
print(json_exfile_updated_path)

/Users/zsk4gm/Desktop/resilience_education/json_files_updated/1-608.json


In [7]:
def load_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data
    
# Example usage
loaded_data = load_json_file(json_exfile_updated_path)
print("Loaded JSON data:", loaded_data)

Loaded JSON data: {'TitleNumber': '1', 'TitleName': 'General Provisions', 'ChapterList': [{'SubtitleNum': '', 'SubtitleName': '', 'PartNum': '', 'PartName': '', 'ChapterNum': '6', 'ChapterName': 'Virginia Coordinate Systems', 'SubPartNum': '', 'SubPartName': '', 'ArticleNum': '', 'ArticleName': '', 'SectionRange': '§ 1-608', 'SectionNumber': '1-608', 'SectionTitle': 'Limitation on use of name of systems', 'SectionText': None, 'Body': '<p>The use of the terms "Virginia Coordinate System of 1927" or "Virginia Coordinate System of 1983" on any map, report of survey, or other document shall be limited to coordinates based on the Virginia coordinate systems as defined in this chapter.</p><p>1946, p. 168; Michie Suppl. 1946, § 2849(7); Code 1950, § 55-295; 1984, c. 726; 2019, c. <a href=\'http://lis.virginia.gov/cgi-bin/legp604.exe?191+ful+CHAP0712\'>712</a>.</p><p class=\'sidenote\'>The chapters of the acts of assembly referenced in the historical citation at the end of this section may not

## Load JSON data in docs

In [8]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import JSONLoader

In [9]:
from bs4 import BeautifulSoup

def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["TitleNumber"] = record.get("TitleNumber")
    metadata["TitleName"] = record.get("TitleName")
    
    chapter_list = record.get("ChapterList", [])
    hrefs = []
    for chapter in chapter_list:
        metadata['ChapterNum'] = chapter.get('ChapterNum')
        metadata['ChapterName'] = chapter.get('ChapterName')
        metadata['ArticleNum'] = chapter.get('ArticleNum')
        metadata['ArticleName'] = chapter.get('ArticleName')
        metadata['SectionNumber'] = chapter.get('SectionNumber')
        metadata['SectionTitle'] = chapter.get('SectionTitle')
        
        # Extract href from Body if available
        body_content = chapter.get("Body", "")
        soup = BeautifulSoup(body_content, 'html.parser')
        links = soup.find_all('a', href=True)
        for link in links:
            hrefs.append(link['href'])
            
    # Convert hrefs list into a single string
    metadata['Hrefs'] = "; ".join(hrefs) 
    
    # Extract all 'Body' contents
    body_contents = [chapter.get("Body", "") for chapter in chapter_list]
    
    return {
        "metadata": metadata,
        "page_content": "\n".join(body_contents)
    }

In [10]:
json_files_test_path = os.path.join(current_directory, 'json_files_test')

loader = DirectoryLoader(
    json_files_test_path,
    glob='*.json',
    loader_cls=JSONLoader,
    loader_kwargs={
        'jq_schema': '.', 
        'content_key': None,
        'metadata_func': metadata_func,
        'text_content': False
    }
)

# Load documents directly
documents = loader.load()

In [11]:
for doc in documents:
    doc.page_content = doc.metadata['page_content']
    del doc.metadata['page_content']  

In [12]:
from langchain.docstore.document import Document
# Function to flatten metadata
def flatten_metadata(document):
    flattened_metadata = document.metadata['metadata']
    return Document(page_content=document.page_content, metadata=flattened_metadata)

# Flatten the metadata in all documents
documents = [flatten_metadata(doc) for doc in documents]

In [13]:
documents[1]

Document(metadata={'source': '/Users/zsk4gm/Desktop/resilience_education/json_files_test/1-3.json', 'seq_num': 1, 'TitleNumber': '1', 'TitleName': 'General Provisions', 'ChapterNum': '1', 'ChapterName': 'Code of Virginia', 'ArticleNum': '', 'ArticleName': '', 'SectionNumber': '1-3', 'SectionTitle': 'Repeal of acts of a general nature', 'Hrefs': 'http://law.lis.virginia.gov/vacode/1-2.1/'}, page_content="<p>All acts and parts of acts of a general nature, in force at the time of the adoption of this Code, shall be repealed from and after February 1, 1950, with such limitations and exceptions as are hereinafter expressed; and all acts and parts of acts of a general nature in force at the time of the adoption of this Code, and in conflict with any of the provisions of this Code which by virtue of § <a href='http://law.lis.virginia.gov/vacode/1-2.1/'>1-2.1</a> become effective on July 1, 1948, shall be repealed from and after July 1, 1948, with such limitations and exceptions as are hereina

## Chunk data in docs

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=300)
all_splits = text_splitter.split_documents(documents)

In [16]:
all_splits[0]

Document(metadata={'source': '/Users/zsk4gm/Desktop/resilience_education/json_files_test/1-2.json', 'seq_num': 1, 'TitleNumber': '1', 'TitleName': 'General Provisions', 'ChapterNum': '1', 'ChapterName': 'Code of Virginia', 'ArticleNum': '', 'ArticleName': '', 'SectionNumber': '1-2', 'SectionTitle': 'Effective date of Code', 'Hrefs': 'http://law.lis.virginia.gov/vacode/1-2.1/'}, page_content="<p>All the provisions of this Code shall be in force on and after February 1, 1950, except as provided in § <a href='http://law.lis.virginia.gov/vacode/1-2.1/'>1-2.1</a>.</p><p>Code 1919, § 6567; R. P. 1948, § 1-2.</p><p class='sidenote'>The chapters of the acts of assembly referenced in the historical citation at the end of this section may not constitute a comprehensive list of such chapters and may exclude chapters whose provisions have expired.</p>")

## Load embedding model

##  Huggingface transformers - perform on search tasks across domains
https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#semantic-search-models

In [17]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "multi-qa-mpnet-base-dot-v1"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/Users/zsk4gm/Desktop/resilience_education/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/zsk4gm/Desktop/resilience_education/env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


## Embed chunks and store in Chroma vector db

In [21]:
from langchain_chroma import Chroma

In [22]:
# Create the file path to the 'embeddings' folder
persist_directory = os.path.join(current_directory, 'embeddings')

# Create the vector store and specify the persist directory
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=hf)

In [23]:
# Process and store embeddings in batches
batch_size_limit = 41000
for i in range(0, len(all_splits), batch_size_limit):
    batch = all_splits[i:i+batch_size_limit]
    vectorstore.add_documents(documents=batch, embedding=hf)

# RAGAS Synthetic Test Set

* generator llm = the component that generates the questions and evolves the question to make it more relevant
* critic llm = component that filters the questions and nodes based on the question and node relevance

In [24]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

class CustomOllama(Ollama):
    def __init__(self, model, callback_manager=None, stop=None, temperature=0.2):
        super().__init__(model=model, stop=stop, temperature=temperature)
        self.callback_manager = callback_manager

# Initialize the CallbackManager
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Create an instance of the CustomOllama class
llm = CustomOllama(
    model="llama3",
    callback_manager=callback_manager
)

In [56]:
critic_llm = Ollama(model="gemma")

In [57]:
# define llm and embeddings
langchain_llm = llm # any langchain LLM instance
langchain_embeddings = hf # any langchain Embeddings instance

In [58]:
# make sure to wrap them with wrappers
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

langchain_llm = LangchainLLMWrapper(langchain_llm)
langchain_embeddings = LangchainEmbeddingsWrapper(langchain_embeddings)
critic_llm = LangchainLLMWrapper(critic_llm)

In [59]:
from ragas.testset.generator import TestsetGenerator

# generator with custom llm and embeddings
generator = TestsetGenerator.from_langchain(
    generator_llm=langchain_llm,
    critic_llm=critic_llm,
    embeddings=langchain_embeddings,
) 

In [60]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# default extractor
from ragas.testset.extractor import KeyphraseExtractor
# default DocumentStore
from ragas.testset.docstore import InMemoryDocumentStore

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=300)
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=langchain_embeddings,
    extractor=keyphrase_extractor
)

In [61]:
from ragas.testset.prompts import (
    context_scoring_prompt,
    evolution_elimination_prompt,
    filter_question_prompt,
)


In [67]:
from ragas.testset.filters import QuestionFilter, EvolutionFilter, NodeFilter

qa_filter = QuestionFilter(langchain_llm, filter_question_prompt)
node_filter = NodeFilter(langchain_llm, context_scoring_prompt=context_scoring_prompt)
evolution_filter = EvolutionFilter(langchain_llm, evolution_elimination_prompt)

In [66]:
distributions = {simple: 0.25, reasoning: 0.25, multi_context: 0.5}

# customise the filters
from ragas.testset.evolutions import ComplexEvolution

for evolution in distributions:
    if evolution.question_filter is None:
        evolution.question_filter = qa_filter
    if evolution.node_filter is None:
        evolution.node_filter = node_filter

    if isinstance(evolution, ComplexEvolution):
        if evolution.evolution_filter is None:
            evolution.evolution_filter = evolution_filter